# Deploy Watson ML (PMML)

Deploys a PMML model to IBM Watson Machine Learning (WML)

In [1]:
!pip install ibm-watson-machine-learning==1.0.45
# PLEASE RESTART YOUR KERNAL AFTER THIS LINE HAS BEEN EXECUTED

  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.7.0-py2.py3-none-any.whl size=72563 sha256=d173f30a94bf0a110bd57afa5f6a7b5e6847d0fc2d6ff85d2a2dd878a176a407
  Stored in directory: c:\users\laptop\appdata\local\pip\cache\wheels\56\14\78\c2fc4fe36e1e7198e4bde8707cf59b838a7be6859c631142f6
  Created wheel for ibm-cos-sdk-core: filename=ibm_cos_sdk_core-2.7.0-py2.py3-none-any.whl size=501005 sha256=09a96bd889881b9973c3bf161942c586b8acec176675ecb1528bcfa9b2eb6b58
  Stored in directory: c:\users\laptop\appdata\local\pip\cache\wheels\54\a5\95\ac2068077b0f41719c2a69a03fdfbdd76ad02cdbb0984ad1a7
  Created wheel for ibm-cos-sdk-s3transfer: filename=ibm_cos_sdk_s3transfer-2.7.0-py2.py3-none-any.whl size=88607 sha256=0af9927dd060bbfb03a5af59e8acce6623c3abf7aba8c9eb297a1f39a73c3199
  Stored in directory: c:\users\laptop\appdata\local\pip\cache\wheels\bf\bf\71\d4b460c36adadb88a24dfd26411c8b4dfa19459d15988c648d
Successfully built ibm-cos-sdk ibm-cos-sdk-core ibm-cos-sdk-s3transfer
  Attempting u

In [1]:
from ibm_watson_machine_learning import APIClient
import logging
import os
import re
import sys

In [2]:
# IBM Cloud API Key https://cloud.ibm.com/iam/apikeys
api_key = os.environ.get('api_key', 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')

# Machine Learning Model Deployment Space https://dataplatform.cloud.ibm.com/ml-runtime/spaces
space = os.environ.get('space', 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')

# IBM Cloud Region (e.g. us-south)
location = os.environ.get('location', 'us-south')

# temporary directory for data
#data_dir = os.environ.get('data_dir',
#                          '../../data/')

In [3]:
parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))

for parameter in parameters:
    logging.warning('Parameter: ' + parameter)
    exec(parameter)

In [4]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

In [5]:
client = APIClient(wml_credentials)

In [6]:
o = client.software_specifications.get_uid_by_name('spark-mllib_2.4')
software_spec_uid = o
client.set.default_space(space)

'SUCCESS'

In [7]:
model_meta_props = {
    client.repository.ModelMetaNames.NAME: 'test_pmml2',
    client.repository.ModelMetaNames.TYPE: "pmml_4.2",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
}

published_model = client.repository.store_model(
    model='model.xml',
    meta_props=model_meta_props,
)

model_uid = client.repository.get_model_uid(published_model)